In [1]:
!pip install tokenizers
!pip install accelerate -U
!pip install transformers[torch]
!pip install torch==1.4.0

ERROR: Could not find a version that satisfies the requirement torch==1.4.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0)
ERROR: No matching distribution found for torch==1.4.0


In [2]:
import pandas as pd
text_data="I am feeling well"

In [3]:

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
punctuations = set(string.punctuation)

def preprocess(text):
  tokens= word_tokenize(text)
  tokens = [token.lower() for token in tokens]
  tokens= [token for token in tokens if token not in stop_words]
  tokens=[token for token in tokens if token not in punctuations]
  return tokens
processed_data = preprocess(text_data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers
tokenizer= Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
tokenizer.decoder = decoders.ByteLevel()

trainer = trainers.BpeTrainer(special_tokens=["<s>","<pad>","</s>","<unk>"])
tokenizer.train_from_iterator(processed_data, trainer=trainer)

encoded_data = tokenizer.encode_batch(processed_data)

In [5]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_steps=500,
    save_steps=2000,
    learning_rate=5e-5,
    warmup_steps=500,
    evaluation_strategy='steps',
    eval_steps=1000,
    load_best_model_at_end=True
)

In [30]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('gpt2', return_dict=True)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

Using pad_token, but it is not set yet.


In [31]:
from transformers import DataCollatorForLanguageModeling, LineByLineTextDataset, Trainer
import torch

# Specify the file path where you want to save the text file
file_path = "data.txt"
str_processed_data=", ".join(processed_data)
# Open the file in write mode and write the string data
with open(file_path, 'w') as file:
    file.write(str_processed_data)
dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path="./data.txt", block_size=256)
data_collator= DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer= Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=3, training_loss=5.6134033203125, metrics={'train_runtime': 11.4143, 'train_samples_per_second': 0.263, 'train_steps_per_second': 0.263, 'total_flos': 6124032000.0, 'train_loss': 5.6134033203125, 'epoch': 3.0})

In [34]:
model.save_pretrained('savemodel')
tokenizer.save_pretrained('savedmodel')

('savedmodel/tokenizer_config.json',
 'savedmodel/special_tokens_map.json',
 'savedmodel/vocab.json',
 'savedmodel/merges.txt',
 'savedmodel/added_tokens.json')

In [37]:
from transformers import pipeline

nlp_task = "sentiment-analysis"
nlp_pipeline=pipeline(nlp_task, model=model, tokenizer=tokenizer)
sentiment=nlp_pipeline("I am feeling good")

The model 'GPT2LMHeadModel' is not supported for sentiment-analysis. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification', 'GPT2ForSequenceCla

KeyError: ignored

ModuleNotFoundError: ignored